In [ ]:
import pickle as pkl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
import json
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open("QBfantasyDF", 'rb') as picklefile: 
    df = pkl.load(picklefile)

In [ ]:
# drop unneccessary columns and put dependent variable column first
finalDF = df.drop(['Name', 'Week', 'OPP'], axis=1)
col = finalDF.columns
col = [col[-2], col[0], col[1], col[2], col[3], col[4], col[5], col[6], col[7], col[8], col[9], col[10], col[12]]
finalDF = finalDF[col]

In [ ]:
# see correlation numbers
finalDF.corr()
#sns.pairplot(finalDF)

In [ ]:
# create variables for linear regression (test all - Lasso works best)
X=finalDF[col[1:]]
y=finalDF['Fantasy Points']
reg = Lasso()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# scale X_train and X_test data
ssX = StandardScaler()
X_train_scaled = ssX.fit_transform(X_train)
X_test_scaled = ssX.transform(X_test)

In [ ]:
# cross validation
scores = cross_val_score(reg, X_train, y_train, cv=10, scoring='r2')
model = reg.fit(X_train, y_train)
scores.mean()

In [ ]:
# check coefficients - which can be removed due to overfitting??
model.coef_

In [ ]:
# eliminate some features
X=finalDF[[col[2], col[3], col[9], col[11]]]
y=finalDF['Fantasy Points']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)
X_train_scaled = ssX.fit_transform(X_train)
X_test_scaled = ssX.transform(X_test)
scores = cross_val_score(reg, X_train, y_train, cv=10, scoring='r2')
model = reg.fit(X_train, y_train)
scores.mean()

In [ ]:
# mess around with parameters and try to find best fits
model = Lasso()
parameters = {'alpha': [1e-20, 1e-16, 1e-12], 'fit_intercept': [True,False]}
grid = GridSearchCV(model, parameters, cv=10, scoring='r2')
grid.fit(X_train_scaled, y_train)
print(grid.cv_results_)
print(grid.best_params_, grid.best_score_)

In [ ]:
# more cross validation
best_lasso = grid.best_estimator_
best_lasso.fit(X_train_scaled, y_train)
scores = cross_val_score(reg, X_train, y_train, cv=10, scoring='r2')
scores.mean()

In [ ]:
# test for different alphas in different degrees of complexity
score = 0
alphas = [1e-15, 1e-12, 1e-9, 1e-6, 1e-3, 1, 1e3, 1e6, 1e9]
degrees = [2, 3, 4, 5, 6, 7, 8, 9]
for alpha, degree in zip(alphas, degrees):
    est = make_pipeline(PolynomialFeatures(bestDegree), Lasso(alpha=bestAlpha))
    est.fit(X_train_scaled, y_train)
    current = est.score(X_test_scaled,y_test)
    print(current)
    if current > score:
        score = current
        bestAlpha = alpha
        bestDegree = degree
print(score, bestAlpha, bestDegree)

In [ ]:
# create function to loop through
def getScore(X_train_scaled, X_test_scaled, y_train, y_test):
    est = make_pipeline(PolynomialFeatures(2), Lasso(alpha=1e-16))
    currentModel = est.fit(X_train_scaled, y_train)
    testScore = est.score(X_test_scaled,y_test)
    trainScore = est.score(X_train_scaled, y_train)
    print(trainScore, testScore)
    return (trainScore, testScore)

In [ ]:
# find which columns work the best as features (up to 3 allowed)
newHigh = 0
for a in range(1,12):
    for b in range(1,12):
        print(a, b)
        for c in range(1,12):
            X=finalDF[[col[a], col[b], col[c]]]
            y=finalDF['Fantasy Points']
            reg = Lasso()
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            X_train_scaled = ssX.fit_transform(X_train)
            X_test_scaled = ssX.transform(X_test)
            current = getScore(X_train_scaled, X_test_scaled, y_train, y_test)
            if current[1] > newHigh and current[1] < 0.99:
                newHigh = current[1]
                print('NEW HIGH')
                print(current)

In [ ]:
# do the same thing as above, except this time use polynomial features (2)
newHigh = 0
for a in range(1,12):
    print(a)
    for b in range(1,12):
        print(b)
        for c in range(1,12):
            X=finalDF[[col[a], col[b], col[c]]]
            y=finalDF['Fantasy Points']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            poly = PolynomialFeatures(2, include_bias=True)
            XTrainScaled = poly.fit_transform(X_train)
            XTestScaled = poly.transform(X_test)
            model = Lasso(alpha=1e-8)
            model.fit(XTrainScaled, y_train)
            trainR = model.score(XTrainScaled, y_train)
            testR = model.score(XTestScaled, y_test)
            if testR > newHigh and testR < 0.99:
                newHigh = testR
                print('NEW HIGH')
            print(trainR, testR)

In [ ]:
# best fit is polynomial with complexity of 2 and features 3, 7, and 11, find best alpha
X=finalDF[[col[3], col[7], col[11]]]
y=finalDF['Fantasy Points']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
poly = PolynomialFeatures(2, include_bias=True)
XTrainScaled = poly.fit_transform(X_train)
XTestScaled = poly.transform(X_test)
myModel = model.fit(XTrainScaled, y_train)
trainR = myModel.score(XTrainScaled, y_train)
testR = myModel.score(XTestScaled, y_test)
print(trainR, testR)

In [ ]:
# graph residuals to look for patterns
predictions = myModel.predict(XScaled)
residuals = predictions - y

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.scatter(range(len(residuals)), residuals)

In [ ]:
# create model using all data points
XScaled = poly.fit_transform(X)
myModel = model.fit(XScaled,y)

In [ ]:
# get final coefficients
coeff_dic = {}
n = 0
for coef_ in myModel.coef_:
    if abs(myModel.coef_[n].round(4)) != 0:
        coeff_dic[poly.get_feature_names([col[3], col[7], col[11]])[n]] = myModel.coef_[n].round(4)
        n+=1
    else:
        n+=1
        
print('Ridge Regression Coefficients\nPolynomial Degree 2\nAlpha = 1960')
print('Intercept:', myModel.intercept_.round(4))
print('Total Non-Zero Coefficients: {}\n'.format(len(coeff_dic)))
print(json.dumps(coeff_dic, indent = 1))